In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display # used to display multiple command outputs in same cell
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OrdinalEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


##### *Loading test and train files*

In [63]:
# Editing the setting to show all the columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [64]:
# Storing train and test csv file path
train_path = "../input/home-data-for-ml-course/train.csv"
test_path = "../input/home-data-for-ml-course/test.csv"

# loading train and test csv files into a panda dataframe
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

### Analyzing train and test data

In [65]:
# Finding number of rows and columns in train and test data
print("Training DataFrame")
display(train_df.shape)
print("Testing DataFrame")
display(test_df.shape)

Training DataFrame


(1460, 81)

Testing DataFrame


(1459, 80)

In [66]:
# Top 5 rows in train and test df
print("Training DataFrame Snippet")
display(train_df.head())
print("Testing DataFrame Snippet")
display(test_df.head())

Training DataFrame Snippet


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Testing DataFrame Snippet


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [67]:
def create_summary(df):
#     Copying the dataframe passed into a different dataframe
    x = df.copy()
    
#     Creating empty dataframe
    summary_df = pd.DataFrame()
    
#     Storing Column names as values
    summary_df["columns"] = list(x.columns.values)
    
#     Adding data type of each column
    summary_df["data_type"] = ""
    
    for i in range(len(summary_df)):
        summary_df.loc[i, ["data_type"]] = x[summary_df.loc[i,"columns"]].dtypes
    
#     Adding number of rows for each columns
    summary_df["#_rows"] = len(x)
    
#     Number of null values in each columns
    summary_df["#_null_values"] = ""
    
    for i in range(len(summary_df)):
         summary_df.loc[i, ["#_null_values"]] = x[summary_df.loc[i,"columns"]].isnull().sum()
    
#     Number of unique values in each columns
    summary_df["#_unique_values"] = ""
    
    for i in range(len(summary_df)):
         summary_df.loc[i, ["#_unique_values"]] = len(x[summary_df.loc[i,"columns"]].unique())
    
#     Returning the summary dataframe
    return summary_df

In [68]:
# Test and train summary
train_summary_df = create_summary(train_df)
test_summary_df = create_summary(test_df)

summary_df = pd.merge(train_summary_df, test_summary_df, how = "outer", on = "columns",
                    suffixes=['_train','_test'])

display(summary_df)

,columns,data_type_train,#_rows_train,#_null_values_train,#_unique_values_train,data_type_test,#_rows_test,#_null_values_test,#_unique_values_test
0,Id,int64,1460,0,1460,int64,1459.0,0,1459
1,MSSubClass,int64,1460,0,15,int64,1459.0,0,16
2,MSZoning,object,1460,0,5,object,1459.0,4,6
3,LotFrontage,float64,1460,259,111,float64,1459.0,227,116
4,LotArea,int64,1460,0,1073,int64,1459.0,0,1106
5,Street,object,1460,0,2,object,1459.0,0,2
6,Alley,object,1460,1369,3,object,1459.0,1352,3
7,LotShape,object,1460,0,4,object,1459.0,0,4
8,LandContour,object,1460,0,4,object,1459.0,0,4
9,Utilities,object,1460,0,2,object,1459.0,2,2


In [69]:
# Description of all numerical columns in train and test data
print("Training DataFrame Numerical Column Description :")
display(train_df.describe())
print("Testing DataFrame Numerical Column Description :")
display(test_df.describe())

Training DataFrame Numerical Column Description :


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


Testing DataFrame Numerical Column Description :


,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1459.000000,1459.000000,1232.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1444.000000,1458.000000,1458.000000,1458.000000,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1457.000000,1457.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.00000,1381.000000,1458.000000,1458.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.580357,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.709141,439.203704,52.619342,554.294925,1046.117970,1156.534613,325.967786,3.543523,1486.045922,0.434454,0.065202,1.570939,0.377656,2.854010,1.042495,6.385195,0.58122,1977.721217,1.766118,472.768861,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705
std,421.321334,42.746880,22.376841,4955.517327,1.436812,1.113740,30.390071,21.130467,177.625900,455.268042,176.753926,437.260486,442.898624,398.165820,420.610226,44.043251,485.566099,0.530648,0.252468,0.555190,0.503017,0.829788,0.208472,1.508895,0.64742,26.431175,0.775945,217.048611,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,407.000000,0.000000,0.000000,407.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.00000,1895.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,1825.500000,20.000000,58.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,0.000000,219.250000,784.000000,873.500000,0.000000,0.000000,1117.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.00000,1959.000000,1.000000,318.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000
50%,2190.000000,50.000000,67.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.500000,0.000000,460.000000,988.000000,1079.000000,0.000000,0.000000,1432.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,0.00000,1979.000000,2.000000,480.000000,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,753.500000,0.000000,797.750000,1305.000000,1382.500000,676.000000,0.000000,1721.000000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.00000,2002.000000,2.000000,576.000000,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,1526.000000,2140.000000,5095.000000,5095.000000,1862.000000,1064.000000,5095.000000,3.000000,2.000000,4.000000,2.000000,6.000000,2.000000,15.000000,4.00000,2207.000000,5.000000,1488.000000,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000


In [70]:
def buggy_column(x, y, a, b):
#     Storing arguements into a variable
    identifier = x
    predict = y
    train_df = a
    test_df = b
    
#     Finding column name in each data set
    train_df_columns = train_df.columns.values.tolist()
    test_df_columns = test_df.columns.values.tolist()
    
#     Checking if supplied identifier is in train and test dataframe
    if identifier not in train_df_columns:
        return print("Given identifer column is not present in train dataframe")
    if identifier not in test_df_columns:
        return print("Given identifer column is not present in test dataframe")

#     Checking if predict column is present in the train_df
    if predict not in train_df_columns:
        return print("Given column to predict is not present in train dataframe")
    
#     Checking if all the columns of train_df is present in test_df
    not_present_columns = []
    for i in train_df_columns:
        if (i not in test_df_columns) and (i != predict):
            not_present_columns.append(i)
    if len(not_present_columns) > 0:
        print("Above train dataframe columns are missing from test dataframe")
        print(not_present_columns)

#     Checing if all the values in each column of test are present in train data
    not_present_columns.append(predict)
    not_present_columns.append(identifier)
    buggy_columns = []
    for i in train_df_columns:
        if i not in not_present_columns:
            for y in [a for a in test_df[i].unique() if a==a]:
                if y not in [b for b in train_df[i].unique() if b==b]:
                    buggy_columns.append(i)
    
    print("Below columns in test dataframe have extra values which are not present in the train data")
    print("Be extra careful while using decision trees")
    return np.unique(np.array(buggy_columns))

In [71]:
# Identify the columns in test and train data for whom values dont match

# Put data into function in following manner :
# buggy_column("identifier", "predicter", train_df, test_df)

identifier = "Id"
predicter = "SalePrice"

buggy_column(identifier, predicter, train_df, test_df)

Below columns in test dataframe have extra values which are not present in the train data
Be extra careful while using decision trees


array(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'EnclosedPorch', 'Fireplaces', 'FullBath',
       'GarageArea', 'GarageCars', 'GarageYrBlt', 'GrLivArea', 'LotArea',
       'LotFrontage', 'LowQualFinSF', 'MSSubClass', 'MasVnrArea',
       'MiscVal', 'OpenPorchSF', 'PoolArea', 'ScreenPorch',
       'TotRmsAbvGrd', 'TotalBsmtSF', 'WoodDeckSF', 'YearBuilt'],
      dtype='<U13')

In [72]:
# Verifying for how many houses remodelling was done after selling the house, as this may lead to data leakage

# x = pd.dataframe()
x = train_df["YearRemodAdd"] > train_df["YrSold"]

x.sum()

1

#### Defining columns to be used, and creating predictors(x) and to be predicted(y) dataframes

In [153]:
# # Skipped columns with reasons

# Columns removed in 1st iteration
# ["Alley", "FireplaceQu", "PoolQC", "Fence", "MiscFeature", "MiscVal"]

In [154]:
# Defining columns to be used in training dataframe

cols_to_use = [
# # 'Id', - This is the identifier
'MSSubClass',
'MSZoning',
'LotFrontage',
'LotArea',
'Street',
# 'Alley', - Lot of null values
'LotShape',
'LandContour',
'Utilities',
'LotConfig',
'LandSlope',
'Neighborhood',
'Condition1',
'Condition2',
'BldgType',
'HouseStyle',
'OverallQual',
'OverallCond',
'YearBuilt',
'YearRemodAdd',
'RoofStyle',
'RoofMatl',
'Exterior1st',
'Exterior2nd',
'MasVnrType',
'MasVnrArea',
'ExterQual',
'ExterCond',
'Foundation',
'BsmtQual',
'BsmtCond',
'BsmtExposure',
'BsmtFinType1',
'BsmtFinSF1',
'BsmtFinType2',
'BsmtFinSF2',
'BsmtUnfSF',
'TotalBsmtSF',
'Heating',
'HeatingQC',
'CentralAir',
'Electrical',
'1stFlrSF',
'2ndFlrSF',
'LowQualFinSF',
'GrLivArea',
'BsmtFullBath',
'BsmtHalfBath',
'FullBath',
'HalfBath',
'BedroomAbvGr',
'KitchenAbvGr',
'KitchenQual',
'TotRmsAbvGrd',
'Functional',
'Fireplaces',
# 'FireplaceQu', - Lot of null values
'GarageType',
'GarageYrBlt',
'GarageFinish',
'GarageCars',
'GarageArea',
'GarageQual',
'GarageCond',
'PavedDrive',
'WoodDeckSF',
'OpenPorchSF',
'EnclosedPorch',
'3SsnPorch',
'ScreenPorch',
'PoolArea',
# 'PoolQC', - Lot of null values
# 'Fence', - Lot of null values
# 'MiscFeature', - Lot of null values
# 'MiscVal', - Because we are not using MiscFeature
'MoSold',
'YrSold',
'SaleType',
'SaleCondition',
# # 'SalePrice' - This is what we will predict
]

In [155]:
# Creating x and y dataframes from train and test data

train_x = train_df[cols_to_use]
train_y = train_df[predicter]
test_x = test_df[cols_to_use]

### *Column Transformation*

#### Value imputation

In [159]:
# All the types of data present in the dataframe
train_x.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [160]:
"""
When working with Pipelines, while creating a Column transformer it’s suggested to pass index
of columns rather than its name because after transformation it’s converted into Numpy Array
and the array does not have any column names
"""

# Identifying numerical columns
# numerical_cols = list(train_x.select_dtypes(include=['number']).columns)
numerical_cols = [train_x.columns.get_loc(i) for i in train_x.select_dtypes(include=['number']).columns]

# Identifying categorical columns
# categorical_cols = list(train_x.select_dtypes(include=['object']).columns)
categorical_cols = [train_x.columns.get_loc(i) for i in train_x.select_dtypes(include=['object', 'bool']).columns]

# Verifying if all the columns are categorized or not
print("Number of columns in master test dataframe = " + str(len(train_x.columns)))
print("Number of columns in created num + cat column lists = " 
      + str(len(numerical_cols) + len(categorical_cols)))
print("Above both values should be equal")

Number of columns in master test dataframe = 73
Number of columns in created num + cat column lists = 73
Above both values should be equal


In [161]:
# Value imputation for numerical data
numerical_imputer = SimpleImputer(strategy = "mean")

# Value imputation for categorical data
categorical_imputer = SimpleImputer(strategy = "most_frequent")

# Bundle value imputation for numerical and categorical data in a column tranformer
impute_preprocessor = ColumnTransformer([("num", numerical_imputer, numerical_cols),
        ("cat", categorical_imputer, categorical_cols)],
    remainder = "passthrough")

#### Encoding values of object type columns in Train and Test dataframe

In [169]:
# Creating ordinal encoder object
ordinal_encoder = OrdinalEncoder()

# Bundle value imputation for numerical and categorical data, and label encoding for categorical data
encode_preprocessor = ColumnTransformer([("encoder", ordinal_encoder, categorical_cols)],
    remainder = "passthrough")

# # Fiting ordinal encoder with the categorical columns in train data
# train_x[categorical_cols] = ordinal_encoder.fit_transform(train_x[categorical_cols])

# # transforming the categorical columns in test data
# test_x[categorical_cols] = ordinal_encoder.transform(test_x[categorical_cols])

#### Defining Models

In [170]:
# Defining all the models to be used

# Decision tree model
dt_model = DecisionTreeRegressor(random_state=1)

# Random forest model
rf_model = RandomForestRegressor(random_state=1)

# XG boost model
xgb_model = XGBRegressor()

#### Creating pipeline

In [172]:
# defining all the pipelines to be used

# Decision tree pipeline
dt_pipeline = Pipeline(steps=[('impute_preprocessor', impute_preprocessor),
                              ('encode_preprocessor', encode_preprocessor),
                              ('dt_model', dt_model)])
# Fitting data to decision tree pipeline
dt_pipeline.fit_transform(train_x, train_y)

# Random forest pipeline
rf_pipeline = Pipeline(steps=[('impute_preprocessor', impute_preprocessor),
                              ('encode_preprocessor', encode_preprocessor),
                              ('rf_model', rf_model)])
# Fitting data to random forest pipeline
rf_pipeline.fit_transform(train_x, train_y)

# XG boost pipeline
xgb_pipeline = Pipeline(steps=[('impute_preprocessor', impute_preprocessor),
                              ('encode_preprocessor', encode_preprocessor),
                              ('xgb_model', xgb_model)])
# Fitting data to XG boost pipeline
xgb_pipeline.fit_transform(train_x, train_y)

# Display Pipeline
# from sklearn import set_config
# set_config(display='diagram')

AttributeError: This 'Pipeline' has no attribute 'transform'

#### Using MAE and cross validation to identify the best model

In [136]:
# # Cross validation for decision tree
# dt_scores = -1 * cross_val_score(dt_pipeline, train_x, train_y,
#                               cv=5,
#                               scoring='neg_mean_absolute_error')

# print("Cross value MAE for decision tree = ", dt_scores)

# # Cross validation using random forest 
# rf_scores = -1 * cross_val_score(rf_pipeline, train_x, train_y,
#                               cv=5,
#                               scoring='neg_mean_absolute_error')

# print("Cross value MAE for random forest = ", rf_scores)

# # Cross validation using XG boost 
# xgb_scores = -1 * cross_val_score(xgb_pipeline, train_x, train_y,
#                               cv=5,
#                               scoring='neg_mean_absolute_error')

# print("Cross value MAE for XG boost = ", xgb_scores)

# # We can use GridSearchCV through which we can use cross_validation and also tune the modelling parameters

In [138]:
# Finding number of rows in the train data frame
num_rows = len(train_x)

# Cross validation Grid search for decision tree
dt_gs = GridSearchCV(dt_pipeline,
                  param_grid = {'dt_model__max_depth': range(1, num_rows, num_rows//100)},
                  cv=5,
                  scoring='neg_mean_squared_error')

dt_gs.fit(train_x, train_y)

# print("Best decision tree parameter :", dt_gs.best_params_)
print("best cross value MAE for decision tree = ", -dt_gs.best_score_)

# Cross validation Grid search using random forest 
dt_rf = GridSearchCV(rf_pipeline,
                  param_grid = {'rf_model__max_depth': range(1, num_rows, num_rows//100)},
                  cv=5,
                  scoring='neg_mean_squared_error')

dt_rf.fit(train_x, train_y)

# print("Best random forest parameter :", dt_rf.best_params_)
print("best cross value MAE for random forest = ", -dt_rf.best_score_)

# Cross validation Grid search using XG boost 
dt_xgb = GridSearchCV(xgb_pipeline,
                  param_grid = {'xgb_model__max_depth': range(1, num_rows, num_rows//100)},
                  cv=5,
                  scoring='neg_mean_squared_error')

dt_xgb.fit(train_x, train_y)

# print("Best XG boost parameter :", dt_xgb.best_params_)
print("best cross value MAE for XG boost = ", -dt_xgb.best_score_)

AttributeError: 'GridSearchCV' object has no attribute 'fit_transform'